In [1]:
!pip install zenml zenml[server]

     ---------------------------------------- 8.2/8.2 MB 10.6 MB/s eta 0:00:00
     ---------------------------------------- 43.7/43.7 kB 2.1 MB/s eta 0:00:00
     -------------------------------------- 209.8/209.8 kB 6.2 MB/s eta 0:00:00
     ------------------------------------- 525.6/525.6 kB 16.6 MB/s eta 0:00:00
  Using cached docker-6.1.3-py3-none-any.whl (148 kB)
     -------------------------------------- 207.3/207.3 kB 6.2 MB/s eta 0:00:00
     ---------------------------------------- 2.6/2.6 MB 5.8 MB/s eta 0:00:00
     ---------------------------------------- 67.8/67.8 kB ? eta 0:00:00
     ---------------------------------------- 100.3/100.3 kB ? eta 0:00:00
  Using cached click-8.1.3-py3-none-any.whl (96 kB)
     -------------------------------------- 152.9/152.9 kB 9.5 MB/s eta 0:00:00
     ---------------------------------------- 2.1/2.1 MB 12.4 MB/s eta 0:00:00
     ---------------------------------------- 95.4/95.4 kB 5.7 MB/s eta 0:00:00
     -------------------------

ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\joshl\\anaconda3\\Lib\\site-packages\\~crypt\\_bcrypt.pyd'
Consider using the `--user` option or check the permissions.



In [1]:
from datasets import load_dataset

dataset = load_dataset("harpreetsahota/LI_Learning_RAG_Eval_Set")

In [2]:
from zenml import pipeline, step
from typing import Annotated
import logging

c:\Users\joshl\anaconda3\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [1]:
!zenml init

Note: NumExpr detected 20 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
NumExpr defaulting to 8 threads.
Initializing the ZenML global configuration version to 0.57.0
⠋ Initializing ZenML repository at c:\Users\joshl\Desktop\JLDEVTECH\JL 
REPO\Scripts\zenml_rag.
Creating database tables

⠙ Initializing ZenML repository at c:\Users\joshl\Desktop\JLDEVTECH\JL 
REPO\Scripts\zenml_rag.

⠹ Initializing ZenML repository at c:\Users\joshl\Desktop\JLDEVTECH\JL 
REPO\Scripts\zenml_rag.

⠼ Initializing ZenML repository at c:\Users\joshl\Desktop\JLDEVTECH\JL 
REPO\Scripts\zenml_rag.

⠴ Initializing ZenML repository at c:\Users\joshl\Desktop\JLDEVTECH\JL 
REPO\Scripts\zenml_rag.
Creating default workspace 'default' ...
Creating default stack in workspace default...

⠦ Initializing ZenML repository at c:\Users\joshl\Desktop\JLDEVTECH\JL 
REPO\Scripts\zenml_rag.

⠧ Initializing ZenML repository at c:\Users\joshl\Desktop\JLDEVTECH\JL 
REPO\Scripts\zenml_rag.

⠇ Initializing Z

C:\Users\joshl\anaconda3\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [3]:
logger = logging.getLogger()
logger.setLevel(logging.INFO)
logger.info("Logging set up.")

Logging set up.


In [4]:
from pprint import pprint
pprint(
    'Question: ' + dataset['train']['question'][0] + '\n' +
    '####################\n'
    'Context: ' + dataset['train']['context'][0] + '\n' +
    '####################\n'
    'Answer: ' + dataset['train']['answer'][0]
)

('Question: How can I leverage coding to multiply my efforts without needing '
 'permission or money from others?\n'
 '####################\n'
 'Context: This includes books, media, movies, and code. Code is probably the '
 'most powerful form of permissionless leverage. All you need is a computeryou '
 'dont need anyones permission. Forget rich versus poor, white-collar versus '
 'blue. Its now leveraged versus un-leveraged. The most interesting and the '
 'most important form of leverage is the idea of products that have no '
 'marginal cost of replication. This is the new form of leverage. This was '
 'only invented in the last few hundred years. It started with the printing '
 'press. It accelerated with broadcast media, and now its really blown up with '
 'the internet and with coding. Now, you can multiply your efforts without '
 'involving other humans and without needing money from other humans. This '
 'book is a form of leverage. Long ago, I would have had to sit in a lecture

In [100]:
def format_prompt(question, answer, context) -> Annotated[str, 'prompt']:
    prompt = f"""You are a professional QA accuracy judge.
        Using the context as your ground truth, determine if 
        the following question answer pair is accurate or not.
        <QA Pair>
        {question}
        ########
        {answer}
        </QA Pair>
        <Context>
        {context}
        </Context>
        Reply with only the word "True" or "False":
        Answer is factual:
        reply: "True"
        Answer is not factual:
        reply: "False"
    """
    logger.info(f"Prompt: {prompt}")
    return prompt

In [6]:
from anthropic import AnthropicBedrock

client = AnthropicBedrock()

In [116]:
df = dataset['train'].to_pandas()
df.head(2)

,question,context,answer,question_groundedness_score,question_groundedness_score_rationale
0,How can I leverage coding to multiply my effor...,"This includes books, media, movies, and code. ...",To leverage coding for permissionless and cost...,5.0,The context provides a detailed explanation of...
1,How can I leverage code or media to make a for...,and that would have been that. This newest for...,"Focus on mastering coding or media skills, cre...",NaN,The context discusses the shift in how fortune...


In [120]:
# Get df.question_groundedness_score where score is less than 3 to make it interesting
d = df[df['question_groundedness_score'] < 3]
d.tail()

,question,context,answer,question_groundedness_score,question_groundedness_score_rationale
264,How can I trust myself in the face of pain and...,fact for us to tackledeath does not make our f...,Trust yourself by enduring pain and awaiting d...,2.0,The context provided discusses a philosophical...
280,How can I achieve unlimited freedom?,"Freedom Where method is, freedom is not. - The...","To achieve unlimited freedom, you must let go ...",1.0,"The context discusses the nature of freedom, e..."
292,How can I start a trend of martial art films i...,Film-making Longing to make a really good film...,To start a trend of martial art films in the U...,2.0,The context provides a statement of intent to ...
293,How can I improve my skills in film directing?,dealing solely with guns. Here we deal with ev...,Focus on honing your skills in visual storytel...,2.0,The context provided discusses a general prefe...
295,How can I maintain my privacy and simplicity w...,The biggest disadvantage of success. -The bigg...,Maintain your privacy and simplicity by settin...,2.0,The context discusses the disadvantages of suc...


In [107]:
@step
def call_judge(input) -> Annotated[str, "True/False:"]:
    output = client.messages.create(
    temperature=0.0,
    max_tokens=2,
    top_k=3,
    messages=[
        {
            "role": "user",
            "content": input,
        }
    ],
    model="anthropic.claude-3-haiku-20240307-v1:0",
    )
    logger.info(f"Output from judge: {output}")
    return output.content[0].text

In [102]:
from typing import List

def calculate_precision(results: List) -> Annotated[float, "precision"]:
    precision = sum([1 for result in results if result == 'True']) / len(results)
    return precision

In [124]:
@step
def evaluate(questions, answers, contexts) -> Annotated[float, "Precision"]:
    results = []
    prompts = []
    for i in range(0, 16):
        formatted = format_prompt(questions[i], answers[i], contexts[i])
        prompts.append(formatted)
    for i in range(0, 16):
        results.append(call_judge(prompts[i]))
    # calculate precision by counting the number of "True" responses and dividing by the total number of responses
    precision = sum([1 for result in results if result == 'True']) / len(results)
    logger.info(f"Precision: {precision}")
    return precision

In [125]:
@pipeline
def run_pipe():
    questions = df['question'].tolist()[264:280]
    answers = df['answer'].tolist()[264:280]
    contexts = df['context'].tolist()[264:280]

    precision = evaluate(questions, answers, contexts)
    logger.info(f"Precision: {precision}")

run_pipe()

Precision: <zenml.steps.entrypoint_function_utils.StepArtifact object at 0x000001EB325C8460>
Initiating a new run for the pipeline: run_pipe.
Registered new version: (version 10).
Executing a new run.
Using user: default
Using stack: default
  artifact_store: default
  orchestrator: default
You can visualize your pipeline runs in the ZenML Dashboard. In order to try it locally, please run zenml up.
Step evaluate has started.
Prompt: You are a professional QA accuracy judge.
        Using the context as your ground truth, determine if 
        the following question answer pair is accurate or not.
        <QA Pair>
        How can I trust myself in the face of pain and death?
        ########
        Trust yourself by enduring pain and awaiting death without resignation or eagerness. Face both with courage and resilience, knowing that you have the strength to persevere. Farewell.
        </QA Pair>
        <Context>
        fact for us to tackledeath does not make our friend braver to f

PipelineRunResponse(body=PipelineRunResponseBody(created=datetime.datetime(2024, 5, 6, 21, 8, 12, 121162), updated=datetime.datetime(2024, 5, 6, 21, 8, 18, 943322), user=UserResponse(body=UserResponseBody(created=datetime.datetime(2024, 5, 6, 18, 48, 12, 106239), updated=datetime.datetime(2024, 5, 6, 20, 10, 39, 398182), active=True, activation_token=None, full_name='', email_opted_in=False, is_service_account=False, is_admin=True), metadata=None, resources=None, id=UUID('d063d5ea-dbf3-4d20-8cff-d36816fc7415'), permission_denied=False, name='default'), status=<ExecutionStatus.COMPLETED: 'completed'>, stack=StackResponse(body=StackResponseBody(created=datetime.datetime(2024, 5, 6, 18, 48, 11, 725435), updated=datetime.datetime(2024, 5, 6, 18, 48, 11, 725435), user=None), metadata=None, resources=None, id=UUID('9ce1f6d4-5a79-4909-9d11-1da258d062bf'), permission_denied=False, name='default'), pipeline=PipelineResponse(body=PipelineResponseBody(created=datetime.datetime(2024, 5, 6, 21, 8, 

In [ ]:
!zenml up --blocking --port 8502